In [ ]:
!pip install transformers=="4.25.1" sentencepiece=="0.1.97" utoken=="0.1.8" nltk=="3.8.1" datasets=="2.8.0" torch=="1.13.1+cu116" numpy=="1.21.6" tqdm=="4.64.1" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.3 MB/s eta 0:00:00


In [ ]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time
import numpy as np

In [ ]:
from utoken import utokenize
from utoken import detokenize
tok = utokenize.Tokenizer(lang_code='ben')
detok = detokenize.Detokenizer(lang_code='ben')

In [ ]:
# !pip install utoken --quiet
import re
from collections import Counter
from utoken import utokenize
from utoken import detokenize
tok = utokenize.Tokenizer(lang_code='ben')
detok = detokenize.Detokenizer(lang_code='ben')

def repeating_prob(context):
    context_tok = tok.utokenize_string(context.translate(str.maketrans('', '', string.punctuation))).split(" ")
    tok_count =  [context_tok.count(i) for i in context_tok]
    rep_token =  context_tok[tok_count.index(max(tok_count))]
    indices_obj = re.finditer(rep_token,context)
    indices = [index.start() for index in indices_obj]
    indices_diff = [t - s for s, t in zip(indices, indices[1:])]
    if len(indices_diff)==0:
        return 0
    indices_diff_count = [indices_diff.count(i) for i in indices_diff]
    return max(indices_diff_count)


def repeating_char_prob(context):
    context_tok = tok.utokenize_string(context.translate(str.maketrans('', '', string.punctuation))).split(" ")
    max_char_in_each_tok = [Counter(tok).most_common(1)[0][1] for tok in context_tok]
    return max(max_char_in_each_tok)

In [ ]:
path = Path('./translated_squad/squad2_translated_final_aligned.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

train = {}
train['data'] = []

# Search for each passage, its question and its answer
for gi in range(0,400):
    group = squad_dict['data'][gi]
    for passage in group['paragraphs']:
        context = passage['bangla_context']
        train['data'].append(context)

In [ ]:
train_leng = [len(data) for data in train['data']]
    

In [ ]:
import statistics

In [ ]:
statistics.quantiles(train_leng)

[520.0, 649.0, 850.0]

In [ ]:
repeating_char_prob("বোমা হামলার জন্য একটি বোমা হামলার জন্য একটি বোমা হামলার জন্য")

2

In [ ]:
path = Path('./translated_squad/squad1_translated_final_aligned.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

train = {}
train['data'] = []

# Search for each passage, its question and its answer
for gi in range(0,400):
    group = squad_dict['data'][gi]
    for passage in group['paragraphs']:
        context = passage['bangla_context']
        if repeating_prob(context)>3 or repeating_char_prob(context)>10:
            continue
        for qa in passage['qas']:
            data = {}
            data['title'] = group['title']
            data['context'] = context
            qid = qa['id']
            question = qa['q_tran'].strip() # question
            data['question'] = question
            data['id'] = qid
            if repeating_prob(question)>3 or repeating_char_prob(question)>10:
                continue
            answer_start = []
            answer_text = []
            if len(qa['answers'])==0:
                print(data)
                print(qa['answers'])
            for answer in qa['answers']:
                if answer['align_score'] >= 0.5 and answer['a_tran'] in context:
                    answer_start.append(answer['a_tran_start'])
                    answer_text.append(answer['a_tran'])
            if len(answer_text)!=0:
              data['answers'] = {'answer_start':answer_start,'text':answer_text}
              train['data'].append(data)


out_file = open('./train_sq1.json', "w")
json.dump(train, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
path = Path('./translated_squad/squad1_translated_final_aligned.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

val = {}
val['data'] = []

# Search for each passage, its question and its answer
for gi in range(400,442):
    group = squad_dict['data'][gi]
    for passage in group['paragraphs']:
        context = passage['bangla_context']
        if repeating_prob(context)>3 or repeating_char_prob(context)>10:
            continue
        for qa in passage['qas']:
            data = {}
            data['title'] = group['title']
            data['context'] = context
            qid = qa['id']
            question = qa['q_tran'].strip() # question
            data['question'] = question
            data['id'] = qid
            if repeating_prob(question)>3 or repeating_char_prob(question)>10:
                continue
            answer_start = []
            answer_text = []
            for answer in qa['answers']:
                if answer['align_score'] >= 0.5 and answer['a_tran'] in context:
                    answer_start.append(answer['a_tran_start'])
                    answer_text.append(answer['a_tran'])
            if len(answer_text)!=0:
              data['answers'] = {'answer_start':answer_start,'text':answer_text}
              val['data'].append(data)


out_file = open('./val_sq1.json', "w")
json.dump(val, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
path = Path('./translated_squad/squad1_dev_translated_final_aligned.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

test = {}
test['data'] = []

# Search for each passage, its question and its answer
for gi in range(len(squad_dict['data'])):
    group = squad_dict['data'][gi]
    for passage in group['paragraphs']:
        context = passage['bangla_context']
        if repeating_prob(context)>3 or repeating_char_prob(context)>10:
            continue
        for qa in passage['qas']:
            data = {}
            data['title'] = group['title']
            data['context'] = context
            qid = qa['id']
            question = qa['q_tran'].strip() # question
            data['question'] = question
            data['id'] = qid
            if repeating_prob(question)>3 or repeating_char_prob(question)>10:
                continue
            answer_start = []
            answer_text = []
            for answer in qa['answers']:
                if answer['align_score'] >= 0.5 and answer['a_tran'] in context:
                    answer_start.append(answer['a_tran_start'])
                    answer_text.append(answer['a_tran'])
            if len(answer_text)!=0:
              data['answers'] = {'answer_start':answer_start,'text':answer_text}
              test['data'].append(data)

out_file = open('./test_sq1.json', "w")
json.dump(test, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
path = Path('./translated_squad/squad2_translated_final_aligned.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

train = {}
train['data'] = []

# Search for each passage, its question and its answer
for gi in range(0,400):
    group = squad_dict['data'][gi]
    for passage in group['paragraphs']:
        context = passage['bangla_context']
        if repeating_prob(context)>3 or repeating_char_prob(context)>10:
            continue
        for qa in passage['qas']:
            data = {}
            data['title'] = group['title']
            data['context'] = context
            qid = qa['id']
            question = qa['q_tran'].strip() # question
            data['question'] = question
            data['id'] = qid
            if repeating_prob(question)>3 or repeating_char_prob(question)>10:
                continue
            answer_start = []
            answer_text = []
            if len(qa['answers'])==0:
                print(data)
                print(qa['answers'])
            for answer in qa['answers']:
                if answer['align_score'] >= 0.5 and answer['a_tran'] in context:
                    answer_start.append(answer['a_tran_start'])
                    answer_text.append(answer['a_tran'])

            data['answers'] = {'answer_start':answer_start,'text':answer_text}
            train['data'].append(data)


out_file = open('./train_sq2.json', "w")
json.dump(train, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
path = Path('./translated_squad/squad2_translated_final_aligned.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

val = {}
val['data'] = []

# Search for each passage, its question and its answer
for gi in range(400,442):
    group = squad_dict['data'][gi]
    for passage in group['paragraphs']:
        context = passage['bangla_context']
        if repeating_prob(context)>3 or repeating_char_prob(context)>10:
            continue
        for qa in passage['qas']:
            data = {}
            data['title'] = group['title']
            data['context'] = context
            qid = qa['id']
            question = qa['q_tran'].strip() # question
            data['question'] = question
            data['id'] = qid
            if repeating_prob(question)>3 or repeating_char_prob(question)>10:
                continue
            answer_start = []
            answer_text = []
            for answer in qa['answers']:
                if answer['align_score'] >= 0.5 and answer['a_tran'] in context:
                    answer_start.append(answer['a_tran_start'])
                    answer_text.append(answer['a_tran'])

            data['answers'] = {'answer_start':answer_start,'text':answer_text}
            val['data'].append(data)


out_file = open('./val_sq2.json', "w")
json.dump(val, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
path = Path('./translated_squad/squad2_dev_translated_final_aligned.json')

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

test = {}
test['data'] = []

# Search for each passage, its question and its answer
for gi in range(len(squad_dict['data'])):
    group = squad_dict['data'][gi]
    for passage in group['paragraphs']:
        context = passage['bangla_context']
        if repeating_prob(context)>3 or repeating_char_prob(context)>10:
            continue
        for qa in passage['qas']:
            data = {}
            data['title'] = group['title']
            data['context'] = context
            qid = qa['id']
            question = qa['q_tran'].strip() # question
            data['question'] = question
            data['id'] = qid
            if repeating_prob(question)>3 or repeating_char_prob(question)>10:
                continue
            answer_start = []
            answer_text = []
            for answer in qa['answers']:
                if answer['align_score'] >= 0.5 and answer['a_tran'] in context:
                    answer_start.append(answer['a_tran_start'])
                    answer_text.append(answer['a_tran'])

            data['answers'] = {'answer_start':answer_start,'text':answer_text}
            test['data'].append(data)

out_file = open('./test_sq2.json', "w")
json.dump(test, out_file, indent = 4) # save whole data replace parts later
out_file.close()